In [2]:
import numpy as np
import cv2 as cv
import imgkit
import cvzone
from cvzone.SelfiSegmentationModule import SelfiSegmentation

# Spécifiez le chemin correct vers wkhtmltoimage
# Téléchargez et installez "wkhtmltoimage".  https://wkhtmltopdf.org/downloads.html
config = imgkit.config(wkhtmltoimage='C:\\Program Files\\wkhtmltopdf\\bin\\wkhtmltoimage.exe')

def rgb_to_hex(rgb):
    """Convertit une couleur RGB en code hexadécimal."""
    return '#{:02x}{:02x}{:02x}'.format(rgb[0], rgb[1], rgb[2])

# Charger l'image d'origine
img_orig = cv.imread("board/1.jpeg")

# Calculer la couleur moyenne de l'image
average_color = np.mean(img_orig, axis=(0, 1))
average_color = average_color.astype(int)

# Convertir la couleur moyenne en format hexadécimal
background_color = rgb_to_hex(average_color)

# Définir le contenu HTML
html = f"""
<html>
  <body style="font-family: Arial, sans-serif; color: black; background-color: {background_color}; margin: 20px;">
    <h1 style="text-align: center;">Machine Learning</h1>
   <!-- <img src="https://th.bing.com/th/id/OIP.2TumufLJYnKvtmU6UMWe6wHaE8?rs=1&pid=ImgDetMain" alt="Description of Image" style="display: block; margin-left: auto; margin-right: auto; width: 50%; height: auto;">
    -->
    <p style="font-size: 14px; margin: 20px;">
      Machine learning is a subset of artificial intelligence (AI) focused on building systems that can learn from and make decisions based on data. It involves creating algorithms and models that allow computers to improve their performance on a task over time without being explicitly programmed for each specific task.
    </p>
  </body>
</html>
"""

# Définir les dimensions de l'image générée
width = int(img_orig.shape[1])  # Utiliser la largeur de l'image d'origine
height = int(img_orig.shape[0])  # Utiliser la hauteur de l'image d'origine

# Convertir le HTML en image avec les dimensions spécifiées
try:
    imgkit.from_string(html, 'generated_image1.png', config=config, options={'width': width, 'height': height})
    print("Image générée avec succès.")
except Exception as e:
    print(f"Erreur lors de la génération de l'image : {e}")

# Charger l'image générée
img_gen = cv.imread('generated_image1.png')

# Vérifiez que l'image générée existe
if img_gen is None:
    raise FileNotFoundError("L'image générée n'a pas pu être chargée. Assurez-vous que la génération a réussi.")

# Vérifiez les dimensions des images
print(f"Dimensions de img_orig : {img_orig.shape[:2]}")
print(f"Dimensions de img_gen : {img_gen.shape[:2]}")

# Redimensionner img_gen pour correspondre à img_orig
img_gen_resized = cv.resize(img_gen, (img_orig.shape[1], img_orig.shape[0]))

# Superposer les deux images
board_image = cv.addWeighted(img_orig, 0.4, img_gen_resized, 0.6, 0)

# Afficher l'image combinée
# cv.imshow("Combined Image", board_image)

# Sauvegarder l'image combinée
# cv.imwrite('combined_image.png', board_image)

# Initialiser la capture vidéo
cap = cv.VideoCapture("video/vid.mp4")
if not cap.isOpened():
    print("Error: Video file could not be opened.")
    exit()

w = int(cap.get(cv.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv.CAP_PROP_FPS))

# Initialiser SelfiSegmentation
segmentor = SelfiSegmentation()

photo = cv.imread('board/aa.png')
photo = cv.resize(photo, (w, h))

# Lire et redimensionner l'image source
w1 = 440
h1 = 300
m1 = cv.resize(board_image, (w1, h1))

# Ajuster la taille de la sous-région pour s'adapter à img1
photo[204:204+300, 20:20+440] = m1

while True:
    suc, img = cap.read()
    if not suc:
        print("Error: Frame capture failed.")
        break

    # Supprimer l'arrière-plan
    imgOut = segmentor.removeBG(img, photo, cutThreshold=0.2)
    
    # Empiler les images pour affichage
    imgStacked = cvzone.stackImages([img, imgOut], 2, 1)
    
    # Afficher les images empilées
    cv.imshow("Image stacked", imgStacked)
    
    # Sortir de la boucle sur la touche 'q'
    if cv.waitKey(1) & 0xFF == ord('q'):
        break

# Libérer les ressources
cap.release()
cv.destroyAllWindows()


Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               
Image générée avec succès.
Dimensions de img_orig : (303, 474)
Dimensions de img_gen : (303, 474)
